# Marketing G2

In [204]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import sklearn as sk
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact 
from sklearn.neighbors import kneighbors_graph
import random as rnd
from scipy.sparse import csr_matrix
from ipywidgets import interact

## Import data sets

In [61]:
movies=pd.read_csv(r'C:\Users\PC\Desktop\Analitica 3\Marketing\code\movies_full')
ratings=pd.read_csv(r'C:\Users\PC\Desktop\Analitica 3\Marketing\code\ratings')
users=pd.read_csv(r'C:\Users\PC\Desktop\Analitica 3\Marketing\code\users')

In [20]:
ratings.head(5)

,userId,movieId,rating,date
0,1,1,4.0,2000-07-30 13:45:03
1,1,3,4.0,2000-07-30 13:20:47
2,1,6,4.0,2000-07-30 13:37:04
3,1,47,5.0,2000-07-30 14:03:35
4,1,50,5.0,2000-07-30 13:48:51


## EDA

In [4]:
n_users = ratings.userId.unique().shape[0]
n_items = ratings.movieId.unique().shape[0]
print (str(n_users) + ' users')
print (str(n_items) + ' items')

610 users
9724 items


In [8]:
px.histogram(ratings.rating)
#Los usuarios no les tiran tan duro a las peliculas#

In [16]:
ratings.groupby(['rating'])['userId'].count()

rating
0.5     1370
1.0     2811
1.5     1791
2.0     7551
2.5     5550
3.0    20047
3.5    13136
4.0    26818
4.5     8551
5.0    13211
Name: userId, dtype: int64

In [144]:
URmatrix = pd.pivot_table(ratings, values='rating', index='movieId', columns='userId').fillna(0)
URmatrix

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
rate = URmatrix.values
sparsity = float(len(rate.nonzero()[0]))
sparsity /= (rate.shape[0] * rate.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity))#!!!!!!!!!!!!! Mucha Sparsity; Muchos 0s

Sparsity: 1.70%


In [147]:
#saber cuantas calificaciones tiene cada pelicula y cuantas veces el usuario ha calificado.
usuariovotos = ratings.groupby('movieId')['rating'].agg('count')
peliculavotos = ratings.groupby('userId')['rating'].agg('count')

#¿Cúal es el maximo y mínimo de calificaciones de una película?


print('¿Cúal es el maximo y mínimo de calificaciones de un usuario?')
print(usuariovotos.max())
print(usuariovotos.min())

print('¿Cúal es el maximo y minimo de calificacionesde una pelicula?')
print(peliculavotos.max())
print(peliculavotos.min())

¿Cúal es el maximo y mínimo de calificaciones de un usuario?
329
1
¿Cúal es el maximo y minimo de calificacionesde una pelicula?
2698
20


In [148]:
#Filtra por cantidad de votos y recomendaciones, para reducir la sparcity y tener mas fiabilidad
URmatrix = URmatrix.loc[usuariovotos[usuariovotos > 10].index,:]
URmatrix=URmatrix.loc[:,peliculavotos[peliculavotos > 100].index]
URmatrix

userId,1,4,6,7,10,15,17,18,19,20,...,599,600,601,602,603,605,606,607,608,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,2.5,4.5,3.5,4.0,0.0,...,3.0,2.5,4.0,0.0,4.0,4.0,2.5,4.0,2.5,5.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,...,2.5,4.0,0.0,4.0,0.0,3.5,0.0,0.0,2.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,4.5,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176371,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
177765,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
rate = URmatrix.values
sparsity = float(len(rate.nonzero()[0]))
sparsity /= (rate.shape[0] * rate.shape[1])
sparsity *= 100
print('Sparsity: {:4.2f}%'.format(sparsity)) #MEJORAAAAA!!!!!!

Sparsity: 12.38%


# Recomendatios Systems

## Popularity Based

In [47]:
# Train-Test Split
ratings_train, ratings_test = train_test_split(ratings, test_size = 0.4, random_state=42)
print(ratings_train.shape)
print(ratings_test.shape)

(60501, 4)
(40335, 4)


In [7]:
def popularity(train,movieId,userId):
    train_data_grouped=train.groupby([movieId])[userId].count().reset_index()
    train_data_grouped.rename(columns={userId:'score'},inplace=True)
    train_data_sort=train_data_grouped.sort_values(['score',movieId],ascending=[0,1])
    train_data_sort['Rank']=train_data_sort['score'].rank(ascending=0,method='first')
    popularity_recommendations=train_data_sort.head(10)
    return popularity_recommendations

In [18]:
pop_rec_sys=popularity(ratings,'movieId','userId')

In [19]:
pop_rec_sys

,movieId,score,Rank
314,356,329,1.0
277,318,317,2.0
257,296,307,3.0
510,593,279,4.0
1938,2571,278,5.0
224,260,251,6.0
418,480,238,7.0
97,110,237,8.0
507,589,224,9.0
461,527,220,10.0


Como el sistema de recomendacion basado en popularidad es parcialmente inmutable, solo se obtiene un resultado y no un modelo predictivo, como tal.

In [21]:
pop_rec_sys_final=pd.merge(pop_rec_sys,movies[['movieId','title']],on='movieId', how='left')
pop_rec_sys_final

,movieId,score,Rank,title
0,356,329,1.0,Forrest Gump (1994)
1,318,317,2.0,"Shawshank Redemption, The (1994)"
2,296,307,3.0,Pulp Fiction (1994)
3,593,279,4.0,"Silence of the Lambs, The (1991)"
4,2571,278,5.0,"Matrix, The (1999)"
5,260,251,6.0,Star Wars: Episode IV - A New Hope (1977)
6,480,238,7.0,Jurassic Park (1993)
7,110,237,8.0,Braveheart (1995)
8,589,224,9.0,Terminator 2: Judgment Day (1991)
9,527,220,10.0,Schindler's List (1993)


Shotout to Rogish Ghosh to help us with the popularity based recommendation system.

## Content Based

### KNN

In [39]:
# Se utilizo KNN como algoritmo de aprendizaje no supervisado y su competitividad e indicadores

In [205]:
valUR = csr_matrix(URmatrix.values)
URmatrix.reset_index(inplace=True)

In [206]:
#Se usa con metrica de coseno porque permite calcular la distancia y computacionalmente es más rapida.
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(valUR)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [207]:
#Con el modeloe ntrenado, podemos hacer una funcion para recomendar 4 peliculas, basadas en una ingresada
def get_movie_recommendation(Movie_id = list(movies['title'])):
    n_movies_to_reccomend = 10
    
    if len(Movie_id):
        Movie_id=movies[movies['title']==(Movie_id)]        
        movie_idx= Movie_id.iloc[0]['movieId']
        
        distances , indices = knn.kneighbors(valUR[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),\
                               key=lambda x: x[1])[:0:-1]
        
        recommend_frame = []
        
        for val in rec_movie_indices:
            movie_idx = URmatrix.iloc[val[0]]['movieId']
            idx =movies[movies['movieId'] == movie_idx].index
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        return df.sort_values(by=['Distance'])
    
    else:
        
        return "No movies found. Please check your input"

In [212]:
print(interact(get_movie_recommendation))

interactive(children=(Dropdown(description='Movie_id', options=('Toy Story (1995)', 'Jumanji (1995)', 'Grumpie…

<function get_movie_recommendation at 0x0000025680C68DC0>
